# Extract and Store Daily Market Cap Prices

In [3]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

## Initialize classes

In [4]:
import os
import sys
import configparser
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from wrangler.modules.assets.etl import cryptoMCExtractor as crypto
from utils.modules.etl.loader import noSQLwls as nosql
from utils.modules.etl.loader import filesRW as rw

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    crypto = importlib.reload(crypto)
    nosql = importlib.reload(nosql)
    rw = importlib.reload(rw)

__desc__='get crypto macket capitalization data'

clsNoSQL = nosql.NoSQLWorkLoads(desc=__desc__)
''' optional - if not specified class will use the default values '''

prop_kwargs = {"WRITE_TO_FILE":False,
               "DATA_STORE":"google-storage",   # values = aws-s3, local-fs, google-storage
              }
clsMC = crypto.CryptoMarkets(
    desc=__desc__, **prop_kwargs)
clsRW = rw.FileWorkLoads(desc=__desc__, **prop_kwargs)
print("\nClass initialization and load complete!")

All python packages in rezaware loaded successfully!
All assets-module etl-packages in function-CryptoMarket imported successfully!
All etl-module load-packages in function-mongodbwls imported successfully!
All etl-module load-packages in function-filesRW imported successfully!
mongodbwls Class initialization complete
mongodbwls Class initialization complete
CryptoMarket Class initialization complete

Class initialization and load complete!


## Get and store daily marketcap in DB

In [40]:
from bson.json_util import dumps

_source = 'coingeko'
_data, coll_name = clsMC.get_daily_mc_data(data_owner=_source)
print(len(_data))
print(coll_name)
# print(dumps(_data, indent=2))

coingeko
Total 100 documents, duccessful insert count = 100 & modify count = 0
100
coingeko.2022-11-21


## Archive marketcap collection in GCP

In [4]:
__db_name__ = "tip-marketcap"
# __collection__=coll_name
__collection__="coingeko.2022-11-18"
#__to_fname__ = str(date.today())+".json"
__to_fname__ = "2022-11-18.json"
_source = 'coingeko'
__to_folder__ = _source

archived_data = clsMC.cold_store_daily_mc(
    from_db_name=__db_name__,
    from_db_coll=__collection__,
    to_file_name=__to_fname__,
    to_folder_path=__to_folder__,
)
print("archived %d marketcap archived_data

[Error]Function <save_data_file> 127.0.0.1:27017: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 637c3da90ebdcd68694cc5e7, topology_type: Unknown, servers: [<ServerDescription ('127.0.0.1', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('127.0.0.1:27017: [Errno 111] Connection refused')>]>
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezaware/wrangler/modules/assets/etl/cryptoMCExtractor.py", line 416, in cold_store_daily_mc
    _json_data = json.loads(dumps(_data))
  File "/home/nuwan/workspace/rezaware/utils/modules/etl/load/noSQLwls.py", line 552, in wrapper_converter
    self._documents = list(doc_dict)
  File "/home/nuwan/.local/lib/python3.8/site-packages/pymongo/cursor.py", line 1248, in next
    if len(self.__data) or self._refresh():
  File "/home/nuwan/.local/lib/python3.8/site-packages/pymongo/cursor.py", line 1139, in _refresh
    self.__session = self.__collection.database.client._ensure_session

Traceback (most recent call last):
  File "/home/nuwan/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_8879/2128144461.py", line 9, in <cell line: 9>
    archived_data = clsMC.cold_store_daily_mc(
  File "/home/nuwan/workspace/rezaware/wrangler/modules/assets/etl/cryptoMCExtractor.py", line 431, in cold_store_daily_mc
UnboundLocalError: local variable 'write_data' referenced before assignment

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nuwan/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/nuwan/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/home/nuwan/.local/lib/python3.8/site-p

## Read daily marketcap from DB

In [5]:
import json
from bson.json_util import dumps

__as_type__ = 'dict'
__database__ = "tip-marketcap"
__collection__ = "coingeko.2022-11-18"
__find__ = {'asset.symbol':
            {"$regex" : 'eth'},
           }
clsNoSQL.connect = {'DBAUTHSOURCE':__database__}
print(clsNoSQL.connect)
# clsNoSQL.connect={}
_data = clsNoSQL.read_documents(
    as_type=__as_type__,
    db_name = __database__,
    db_coll=__collection__,
    doc_find={}
)
# print(dumps(_data, indent=2))
_json_data = json.loads(dumps(_data))
_json_data
# _data

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True, authsource='tip-marketcap', authmechanism='SCRAM-SHA-256')


ServerSelectionTimeoutError: 127.0.0.1:27017: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 637c3ebe0ebdcd68694cc5e8, topology_type: Unknown, servers: [<ServerDescription ('127.0.0.1', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('127.0.0.1:27017: [Errno 111] Connection refused')>]>